In [5]:
import os
import cv2
import json
import numpy as np
import pandas
import skimage
import sklearn
import shapefile
import matplotlib.pyplot as plt
import tensorflow as tf
import modules

### load data

In [6]:
# TODO: rescale images to workable size (low priority -- use ben's for now)

# TODO: filter unwanted samples e.g. clouds, outliers, etc. (low priority)

In [9]:
config = modules.run.load_config("cls", tag="w5")

dg = modules.data.DatasetGenerator(config)
dg._setup("kenya")

checkpoints_dir = os.path.join("data", config["base"] + config["tag"], "checkpoints")
if not os.path.isdir(checkpoints_dir):
    os.makedirs(checkpoints_dir)

tensorboard_dir = os.path.join("data", config["base"] + config["tag"], "tensorboard")
if not os.path.isdir(tensorboard_dir):
    os.makedirs(tensorboard_dir)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    tensorboard_dir, 
    histogram_freq=1,
    write_graph=True, 
    write_images=True,
    update_freq=config["batch_size"] * config["tensorboard_freq"]
)

checkpoints_callback = tf.keras.callbacks.ModelCheckpoint(
    f"{checkpoints_dir}/model",
    monitor='val_loss',
    save_best_only=False,
    save_weights_only=True,
    save_freq='epoch',
)

input_layer = tf.keras.layers.Input(
    shape=(
        config["image_size"],
        config["image_size"],
        config["n_channels"])
)

vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_tensor=input_layer,
    pooling="max",
    classes=config["n_classes"]
)

cce_loss = tf.keras.losses.CategoricalCrossentropy()

adam_opt = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])

if config["balance_classes"]:
    class_weight = sklearn.utils.class_weight.compute_class_weight(
        "balanced", 
        list(range(config["n_classes"])), 
        [config["class_enum"][v] for v in dg.dataframes["kenya"]["class"].values]
    )
else:
    class_weight = None
    
vgg16.compile(loss=cce_loss, optimizer=adam_opt)

vgg16.summary()

vgg16.fit_generator(
    train_generator, 
    epochs=config["n_epochs"],
    callbacks=[tensorboard_callback, checkpoints_callback], 
    validation_data=val_generator, 
    validation_steps=len(val_generator) // config["batch_size"],
    class_weight=class_weight
)
